# Marktplaats web scraper voor tweedehands autos

In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [46]:
import datetime
import time

today = datetime.date.today()
year_current = today.year  # huidig jaar

## 1. Create URL Link to scrape
link should also edited to search correct product category and filters

### URL link base format

don't edit unless URL characteristics of marktplaats.nl itself changes.

In [47]:
# URL host
URL_host = "https://www.marktplaats.nl/" 

# URL path should be expanded with following filters
URL_p_prijs_min = "PriceCentsFrom:{prijs}|"  # minimumprijs euros
URL_p_prijs_max = "PriceCentsTo:{prijs}|"  # maximum prijs euros
URL_p_bouwjaar_oudst = "constructionYearFrom:{bouwjaar}|" 
URL_p_bouwjaar_nieuwst = "constructionYearTo:{bouwjaar}|"
URL_p_km_stand_min = "mileageFrom:{km_stand}|"
URL_p_km_stand_max = "mileageTo:{km_stand}|"

url_path_list =[URL_p_prijs_min, URL_p_prijs_max, 
                URL_p_bouwjaar_oudst, URL_p_bouwjaar_nieuwst, 
                URL_p_km_stand_min, URL_p_km_stand_max]

### Functions

#### 1. For the URL link, create strings to append the URL path

In [48]:

def URL_path_format_filters(prijs_min=0, prijs_max=100_000, 
                           bouwjaar_oudst=1900, bouwjaar_nieuwst=year_current, 
                           km_stand_min=0, km_stand_max=1_000_000):
    
    global  URL_p_prijs_min, URL_p_prijs_max, URL_p_bouwjaar_oudst, URL_p_bouwjaar_nieuwst, URL_p_km_stand_min, URL_p_km_stand_max
    
    # format the URL path filters
    URL_p_prijs_min = URL_p_prijs_min.format(prijs = prijs_min*100)  # *100 to convert to cents
    URL_p_prijs_max = URL_p_prijs_max.format(prijs = prijs_max*100)
    URL_p_bouwjaar_oudst = URL_p_bouwjaar_oudst.format(bouwjaar = bouwjaar_oudst)
    URL_p_bouwjaar_nieuwst = URL_p_bouwjaar_nieuwst.format(bouwjaar = bouwjaar_nieuwst)
    URL_p_km_stand_min = URL_p_km_stand_min.format(km_stand = km_stand_min)
    URL_p_km_stand_max = URL_p_km_stand_max.format(km_stand = km_stand_max)
    
    # create single string for the path URL
    formatted_filters = URL_p_prijs_min+URL_p_prijs_max+URL_p_bouwjaar_oudst+URL_p_bouwjaar_nieuwst+URL_p_km_stand_min+URL_p_km_stand_max

    return formatted_filters

# return url path gedeelte mbt. adverteerders type
def url_path_adverteerder(particulier=None, bedrijf=None):
    url_path_part =  'f:10898|'  # gedeelte url-path particulier
    url_path_bed = 'f:10899|'  # bedrijf
    if particulier==True and not bedrijf==True:
        return url_path_part
    elif bedrijf==True and not particulier==True:
        return url_path_bed
    else: 
        return None

# return  url path gedeelte mbt. tekstquery
def url_path_query(string):
    string = string.replace(" ", "+")
    if string =="":
        return string
    else:
        subpath = "q:{text}|".format(text=string)
        return subpath

#### 2. Create final URL depending on page

In [49]:
def URL_link_per_page(pageNum):
    global URL_host, URL_p_category, prefix, url_p_query, url_p_adverteerder, url_p_base_filters
    
    if pageNum <=1:
        URL_finalized = URL_host+URL_p_category+ prefix+url_p_query+url_p_adverteerder+url_p_base_filters
        return URL_finalized
    else:
        page_URL_subPath = "p/{page}/".format(page = pageNum)
        URL_finalized = URL_host+URL_p_category+ page_URL_subPath+ prefix+url_p_query+url_p_adverteerder+url_p_base_filters
    
    return URL_finalized

### Tweak manually for search
handmatig aanpassen afhankelijk van wat u wilt zoeken

In [50]:
url_p_base_filters = URL_path_format_filters()

url_p_adverteerder = url_path_adverteerder(particulier=True)

# query die je als tekst moet typen in het 'zoekvak' (string vorm)
# hou het als empty string "" als je niks wilt.
url_p_query = url_path_query("")

# URL host should be expanded with the following path
URL_p_category = "l/auto-s/"  # zoek per categorie "autos". NIET VERANDEREN


### Create URL link to the correct filtered website

In [51]:
prefix = "#"  # prefix needed before formatting


In [52]:
# create URL link
URL_finalized = URL_link_per_page(0)
print(URL_finalized)
    

https://www.marktplaats.nl/l/auto-s/#f:10898|PriceCentsFrom:0|PriceCentsTo:10000000|constructionYearFrom:1900|constructionYearTo:2023|mileageFrom:0|mileageTo:1000000|


## 2. Scraper
scrape data van marktplaats websites op basis van gecreeerde link

In [53]:
URL_host_base = 'https://www.marktplaats.nl'  # zonder '/' aan einde


In [54]:
# scrape aanbiedingen op een webpage dmv URL webpage
def scrape_marktplaats_autos(URL):
    result = requests.get(URL)
    soup = BeautifulSoup(result.text, 'html.parser')   # marktplaats soup
    webpage = soup.find('ul' , class_='hz-Listings hz-Listings--list-view')
    webpage_aanbiedingen = webpage.find_all('li')
    return webpage_aanbiedingen
    

### Scrape 1 enkele pagina
gebreken: 
- bugfix: href niet altijd gevonden

## 3. Scrape marktplaats webpaginas.

### creeer pandas dataframe voor gescrapete data



In [59]:
df = pd.DataFrame(columns = ['titel', 'bouwjaar', 'km_stand', 'brandstof', 'transmissie', 
                             'prijs', 'URL_aanbieding', 'plaatsing_datum', 
                             'aanbieder_naam', 'aanbieder_locatie'])

df

,titel,bouwjaar,km_stand,brandstof,transmissie,prijs,URL_aanbieding,plaatsing_datum,aanbieder_naam,aanbieder_locatie


In [60]:
URL_host_base = 'https://www.marktplaats.nl'  # zonder '/' aan einde

page_max = 3  # maximum aantal paginas die gescraped worden
page_start = 1  # pagina vanaf waar gescraped wordt.

for page_num in range(page_start, (page_max+1)):
    print('scrape page: ', page_num)
    page_URL = URL_link_per_page(page_num)  # creeer URL voor paginanummer
    
    # ga er van uit dat als pagina niet gevonden is, er niet meer paginas zijn en break loop
    try:
        webpage_aanbiedingen = scrape_marktplaats_autos(page_URL)
    except:
        break 
        
    time.sleep(1) # Sleep 1 sec na URL request, om server niet te overbelasten
    
    
    for aanbod in webpage_aanbiedingen:
        
        # find algemene productinformatie; 
        # titel, bouwjaar, km_stand, brandstof, transmissie, prijs, URL_aanbieding
        try:
            aanbod_main = aanbod.find('a', 'hz-Link hz-Link--block hz-Listing-coverLink')
        except:
            aanbod_main = None
        
        
        # find titel
        try:
            titel = aanbod_main.find('div','hz-Listing-headings u-colorTextSecondary')
            titel = titel.text
        except:
            titel = None
        
        ###################################################################
        
        # find bouwjaar, km_stand, brandstof, transmissie
        # auto eigenschappen HTML container
        try:
            listing_attributes = aanbod_main.find('div', class_='hz-Listing-attributes hz-Text hz-Text--bodyRegularStrong')  
            listing_attributes = listing_attributes.find_all('span')
            if len(listing_attributes) >= 4:  # alle 4 variabelen aanwezig
                bouwjaar = listing_attributes[0].text
                km_stand =  listing_attributes[1].text
                brandstof =  listing_attributes[2].text
                transmissie =  listing_attributes[3].text
            else:
                #gemakzuchtig alle data in listing_attributes in bouwjaar cell stoppen
                bouwjaar = listing_attributes
                km_stand = None
                brandstof = None
                transmissie = None
        except:
            bouwjaar = None
            km_stand = None
            brandstof = None
            transmissie = None
        
        
        # find prijs
        try:
            prijs = aanbod_main.find('div', class_='hz-Listing-price hz-Title hz-Title--title4')
            prijs = prijs.text
        except:
            prijs = None
        
        
        # find URL_aanbieding
        try:
            href_aanbieding = aanbod_main['href']
            URL_aanbieding = URL_host_base+href_aanbieding        
        except:
            print('href not FOUND!!!!!!!!!!!!!!!!!!!!!!!')
            URL_aanbieding = None
        
        ##########################################################

       # find aanbieder, locatie, datum plaatsing aanbieding
        try:
            footer = aanbod.find('div', class_='hz-Listing-footer')  # k
        except:
            footer = None

        # find plaatsing_datum
        try:
            plaatsing_datum = footer.find('div', class_='hz-Listing-listingDate hz-Text hz-Text--bodySmall u-colorTextSecondary')
            plaatsing_datum = plaatsing_datum.text
        except:
            plaatsing_datum = None

        # algemene info aanbieder; aanbieder_naam, aanbieder_locatie 
        try:
            aanbieder_algemeen = footer.find('div', class_='hz-Listing-seller')  
        except:
            aanbieder_algemeen = None

        # find aanbieder_naam
        try:
            aanbieder_naam = aanbieder_algemeen.find('span', class_='hz-Listing-sellerName')
            aanbieder_naam = aanbieder_naam.text
        except:
            aanbieder_naam = None

        # find aanbieder_href
        try:
            aanbieder_href = footer.text
        except:
            aanbieder_href = None

        # find aanbieder_locatie
        try:
            aanbieder_locatie = footer.find('div', class_='hz-Listing-sellerLocation hz-Text hz-Text--bodySmall u-colorTextSecondary')
            aanbieder_locatie = aanbieder_locatie.text
        except:
            aanbieder_locatie = None
            
        ################################################################################
        # append extracted data in pandas df

        row_aanbieding = [titel, bouwjaar, km_stand, brandstof, transmissie, prijs, URL_aanbieding,
                          plaatsing_datum, aanbieder_naam, aanbieder_locatie]
        df.loc[len(df.index)] = row_aanbieding



scrape page:  1
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
scrape page:  2
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
scrape page:  3
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!
href not FOUND!!!!!!!!!!!!!!!!!!!!!!!


In [61]:
df

,titel,bouwjaar,km_stand,brandstof,transmissie,prijs,URL_aanbieding,plaatsing_datum,aanbieder_naam,aanbieder_locatie
0,Peugeot Expert L3 H1 2018 €266 per maand,2018,152.803 km,Diesel,Handgeschakeld,"€ 18.700,-",https://www.marktplaats.nl/v/auto-s/bestelauto...,Vandaag,Action Lease,Heel Nederland
1,Opel Vivaro | 1.6 CDTI 145 pk L2H1 VIP BUS (In...,2018,133.140 km,Diesel,Handgeschakeld,"€ 438,-",https://www.marktplaats.nl/v/auto-s/bestelauto...,Vandaag,Regeljelease.nl,Heel Nederland
2,Aixam Roadline -2008- incl. 3 mnd garantie + o...,2008,47.108 km,Diesel,Automaat,"€ 6.950,-",https://www.marktplaats.nl/v/auto-s/aixam/a144...,Vandaag,Garage Kaal & Buunk,Beek
3,Volkswagen Crafter 35 2.0TDI 140pk L3H3 (Oude ...,2018,148.226 km,Diesel,Handgeschakeld,"€ 356,-",https://www.marktplaats.nl/v/auto-s/bestelauto...,Vandaag,Van der Wal Vans,Langerak
4,Opel Agila 1.2 Sportline 5 Drs Airco / Elek ve...,2008,147.477 km,,Benzine,"€ 3.400,-",https://www.marktplaats.nl/v/auto-s/opel/m1938...,Vandaag,Garage Duin,Haarlem
...,...,...,...,...,...,...,...,...,...,...
97,Audi A6 2.4 V6 130KW Multitronic 2004 Zwart. ...,2004,405.000 km,,Benzine,Bieden,None,Vandaag,A.,Wolvega
98,Dodge Ram 1500 10TH ANNIVERSARY LIMITED | INDI...,2023,69 km,LPG,Automaat,"€ 89.950,-",https://www.marktplaats.nl/v/auto-s/dodge/m194...,Vandaag,RAM Dealer NL Sami Auto,Almelo
99,Ford USA F-150 / Special edition / Harley Davi...,2008,297.060 km,Overige brandstoffen,SUV of Terreinwagen,"€ 15.950,-",https://www.marktplaats.nl/v/auto-s/ford-usa/m...,Vandaag,Verlaan BOAT-STORE,Ruinen
100,Dodge Ram 1500 LIMITED NIGHT® RAMBOX | ALL-IN ...,2023,69 km,LPG,Automaat,"€ 85.950,-",https://www.marktplaats.nl/v/auto-s/dodge/m194...,Vandaag,RAM Dealer NL Sami Auto,Almelo
